# Chapter 2: Emulating RISC-V

To execute our compiled code, we'll be using [`RiscEmu`](https://github.com/AntonLydike/riscemu),
a RISC-V emulator written in Python. It implements the [RISC-V ISA](https://msyksphinz-self.github.io/riscv-isadoc/html/rvi.html#addi),
and allows for extending the ISA with custom functions. Here's a simple example
of an assembly program:

In [ ]:
from riscv.emulator_iop import run_riscv

riscv_0 = """
.text                   # The code section begins with .text
main:                   # main is the entry point to this program
	li	%0, 83            # set %0 to 83
	li	%1, 5             # set %1 to 5
	li	%2, 10            # set %2 to 10
	mul %3 %0 %1          # set %3 to %0 * %1
	add %4 %3 %2          # set %4 to %3 + %2
	print	%4            # use a debug print instruction
	li	a7, 93            # a7 is a special register that can be set to the desired syscall code
	scall                 # call exit syscode
"""

# The code uses unlimited registers, that we call %0, %1, %2, ... for convenience
run_riscv(riscv_0, unlimited_regs=True)

### Extending the RISC-V ISA With an L0 Accelerator

`RiscEmu` allows for extending the ISA with custom instructions. These can be seen
as rough equivalents of custom hardware accelerators for specialised computation.
xDSL can then easily be extended to leverage those instructions.

Let's implement a fused multiply add instruction, and call it fmadd, equivalent to
the following python code:

``` python3
def fmadd(a: int, b: int, c: int) -> int:
    return a * b + c
```

In [ ]:
from riscemu.instructions import InstructionSet, Instruction

# Define a RISC-V ISA extension by subclassing InstructionSet
class RV_fmadd(InstructionSet):
    # each method beginning with instruction_ will be available to the Emulator
    
    def instruction_fmadd(self, ins: Instruction):
        """
        This method defines the semantics of the fmadd instruction. Let's settle at:
        
        rd = (rs1 * rs2) + rs3
        """
        # get all register names from the instruction:
        rd, rs1, rs2, rs3 = (ins.get_reg(i) for i in (0,1,2,3))
        
        # we can access the cpu registers through self.regs
        
        # we can set a register value using self.regs.set(name: str, value: Int32)
        self.regs.set(
            rd,
            (self.regs.get(rs1) * self.regs.get(rs2)) + self.regs.get(rs3)
        )

riscv_1 = """
.text                   # The code section begins with .text
main:                   # main is the entry point to this program
	li	%0, 83            # set %0 to 83
	li	%1, 5             # set %1 to 5
	li	%2, 10            # set %2 to 10
  fmadd %3 %0 %1 %2     # set %3 to %0 * %1 + %2
	print	%3              # use a debug print instruction
	li	a7, 93            # a7 is a special register that can be set to the desired syscall code
	scall                 # call exit syscode
"""

run_riscv(riscv_1, extensions=(RV_fmadd,), unlimited_regs=True)